In [ ]:
import pandas as pd
import numpy as np
df= pd.read_csv('master_sample .csv')
#Before cleaning data, cnduct exploratory data analysis by pandas profiling
pip install ydata-profiling
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Profiling Report")
profile
#Output shoes the highly correlated variables, missing values, duplicated values, variable types, whether numeric or categorical, great way for understanding the data

#Removing leading and trailing whitespaces from all column names
df.columns = df.columns.str.strip()
#Remove null values 
df.dropna()
df.dropna(subset=['Transaction Date (Pacific Time)'], inplace=True)

#Crate a new data frame with charger stations that will be analyzed for the project
station_id= ['DEP_26WARD_2_L3','DEP_26WARD_1_L3','DSNY_SI2_DCFC1','DSNY_SI2_DCFC2', 'DSNY_MN2_DCFC2','DSNY_MN2_DCFC1', 
             'DSNY_MN05_DCFC1','DSNY_MN05_DCFC2','DPROCBRZAC_1_L3',' DPROCBRZAC_2_L3', 'DEP_OWLSHD_1_L3','DEP_OWLSHD_2_L3',
            'DEPNTWNCRK_3_L3','DEPNTWNCRK_5_L3', 'NYPD_CADET_1_L3','NYPD_CADET_2_L3', 'DEP_RHWWTP_1_L3','DEP_RHWWTP_2_L3', 
             'DCAS_KNTAV_1_L3','DCAS_KNTAV_2_L3', 'DEPOAKWWTP_1_L3','DEPOAKWWTP_2_L3', 'DOTHPRASPH_1_L3','DOTHPRASPH_2_L3',
             'OCME_26ST_1_L3','OCME_26ST_2_L3', 'DCASCSTHSE_1_L3','DCASCSTHSE_2_L3','DCASCSTHSE_3_L3']

df= df[df['Station Name'].isin(station_id)]

#Create a single column for charger ports
df['Station Name']= df['Station Name'].replace(['DEP_26WARD_2_L3','DEP_26WARD_1_L3'], 'DEP 26th Ward WWTP')
df['Station Name']= df['Station Name'].replace(['DSNY_SI2_DCFC1','DSNY_SI2_DCFC2'], 'DSNY SI2')
df['Station Name']= df['Station Name'].replace(['DSNY_MN2_DCFC2','DSNY_MN2_DCFC1'], 'DSNY MN 2')
df['Station Name']= df['Station Name'].replace(['DSNY_MN05_DCFC1','DSNY_MN05_DCFC2'], 'DSNY MN 05')
df['Station Name']= df['Station Name'].replace(['DPROCBRZAC_1_L3',' DPROCBRZAC_2_L3'], 'DPR Ocean Breeze')
df['Station Name']= df['Station Name'].replace(['DEP_OWLSHD_1_L3','DEP_OWLSHD_2_L3'], 'DEP Owl’s Head WWTP')
df['Station Name']= df['Station Name'].replace(['DEPNTWNCRK_3_L3','DEPNTWNCRK_5_L3'], 'DEP Newtown Creek WWTP')
df['Station Name']= df['Station Name'].replace(['NYPD_CADET_1_L3','NYPD_CADET_2_L3'], 'NYPD Cadet Academy ')
df['Station Name']= df['Station Name'].replace(['DEP_RHWWTP_1_L3','DEP_RHWWTP_2_L3'], 'DEP Red Hook WWTP')
df['Station Name']= df['Station Name'].replace(['DCAS_KNTAV_1_L3','DCAS_KNTAV_2_L3'], 'DCAS Kent Ave')
df['Station Name']= df['Station Name'].replace(['DEPOAKWWTP_1_L3','DEPOAKWWTP_2_L3'], 'DEP Oakwood Beach WWTP')
df['Station Name']= df['Station Name'].replace(['DOTHPRASPH_1_L3','DOTHPRASPH_2_L3'], 'DOT Harper St')
df['Station Name']= df['Station Name'].replace(['OCME_26ST_1_L3','OCME_26ST_2_L3'], 'OCME 26th St')
df['Station Name']= df['Station Name'].replace(['DCASCSTHSE_1_L3','DCASCSTHSE_2_L3','DCASCSTHSE_3_L3'], 'DCAS Storehouse ')

#Extract the hour from the Start Date column adn create a new column
df['Start Date'] = pd.to_datetime(df['Start Date])

#Create a mapping dictionary for hour labels
hour_labels = {
     0:' 12 AM', 1: '1 AM', 2: '2 AM', 3: '3 AM', 4: '4 AM', 5: '5 AM',
     6: '6 AM', 7: '7 AM', 8: '8 AM', 9: '9 AM', 10: '10 AM', 11: '11 AM',
         12: '12 PM', 13: '1 PM', 14: '2PM', 15: '3 PM', 16: '4PM', 17: '5 PM',
         18: '6 PM', 19: '7 PM', 20: '8 PM', 21: '9 PM', 22: '10 PM', 23: '11 PM'
}

#Replace the hour values with the hour labels
df['StartDate_Hour'] = df['StartDate_Hour'].map(hour_labels)
df['StartDate_Minute'] = df['Start Date'].dt.minute
df['StartDate_Hour'] = df['Start Date'].dt.second
df['StartDate_Hour'] = df['Start Date'].dt.month
df['StartDate_Hour'] = df['Start Date'].dt.day
df['StartDate_Hour'] = df['Start Date'].dt.year

#Extract the hour, minute, second from the'Charging time (hh:mm:ss)' column and create a new column
df['Charging time (hh:mm:ss)'] = pd.to_datetime(df['Charging time (hh:mm:ss)])
df['ChargingTime_Hour'] = df['Charging time (hh:mm:ss)'].dt.hour
df['ChargingTime_Minute'] = df['Charging time (hh:mm:ss)'].dt.minute
df['ChargingTime_Second'] = df['Charging time (hh:mm:ss)'].dt.second

df['Total Duration (hh:mm:ss)']=pd.to_timedelta(df['Total Duration (hh:mm:ss)'])

# Create a new column for total duration in hours
df['Total Duration (hours)'] = df['Total Duration (hh:mm:ss)'].dt.total_seconds() / 3600

# Split into days, hours, and minutes
df['Days'] = df['Total Duration (hours)'] // 24
df['Hours'] = df['Total Duration (hours)'] % 24
df['Minutes'] = df['Total Duration (hh:mm:ss)'].dt.components.minutes


#Create bins for Start SOC and SOC Columns
#Convert the 'Start SOC' and 'End SOC' columns from strings to numeric values (floats)
df['Start SOC'] = df['Start SOC'].str.rstrip('%').astype(float)
df['End SOC']= df['End SOC'].str.rstrip('%').astype(float)

#Assign the bins to new columns in the DataFrame
df['StartSOC_bin']= pd.cut(df['Start SOC'], bins = [1.0, 20.0, 40.0, 60.0, 80.0, 96.0])
df['EndSOC_bin']= pd.cut(df['EndSOC_bin'], bins= [6.0, 20.0, 40.0, 60.0, 80.0,100.0])

#read new data to a csv file
df.to_csv('final_df.csv, index=True)

#For number of unique transaction number
df['Transaction ID'].nunique()
#For number of unique User ID number
df['User ID'].nunique()

df_2=df.groupby(['Station Name','StartSOC_bin','EndSOC_bin', 'StartDate_Hour'])['Energy (kWh)', 'Total_Plug_In_Duration_hours', 'Total_Charging_Time(Hour)'].mean()
df_2=df.groupby(['Station Name','StartSOC_bin','EndSOC_bin', 'StartDate_Hour'])['Transaction ID'].nunique()
          

In [ ]:
#Visuals
#Avg. Energy Consumption by User Type
all_users= df[df['Station Name'].isin(['DCAS Storehouse', 'DPR Ocean Breeze', 'NYPD Cadet Academy'])]
city_users=df[df['Station Name'].isin(['DCAS Kent Ave', 'DSNY MN 2', 'DOT Harper St',
       'DSNY SI2', 'OCME 26th St.',
       'DEP Oakwood Beach WWTP', 'DSNY MN 05',
       'DEP Red Hook WWTP', 'DEP Newtown Creek WWTP',
       'DEP 26th Ward WWTP', "DEP Owl's Head WWTP"])]
all_users_avg= all_users.groupby('StartDate_Hour')['Energy (kWh)'].mean()
city_users_avg= city_users.groupby('StartDate_Hour')['Energy (kWh)'].mean()
hours=['12:00 AM','1:00 AM','2:00 AM','3:00 AM','4:00 AM','5:00 AM','6:00 AM','7:00 AM','8:00 AM','9:00 AM','10:00 AM'
       ,'11:00 AM','12:00 PM', '1:00 PM', '2:00 PM', '3:00 PM', '4:00 PM', '5:00 PM', '6:00 PM', '7:00 PM', '8:00 PM', '9:00 PM', '10:00 PM', '11:00 PM']
max_all_users=all_users_avg.max()  
max_city_users=city_users_avg.max()

all_users_avg= all_users_avg.reindex(hours)
city_users_avg= city_users_avg.reindex(hours)

plt.figure(figsize=(11,8))
plt.plot(all_users_avg.index, all_users_avg.values, label= 'All Users', color='darkcyan')
plt.plot(city_users_avg.index, city_users_avg.values, label= 'City Users', color='midnightblue')

plt.annotate(f'Highest Energy Consumption around 4am: {max_all_users:.2f} kWh', xy=(all_users_avg.idxmax(), max_all_users),
             xytext=(all_users_avg.idxmax(), max_all_users ),
             fontsize=10)
plt.annotate(f'Highest Energy Consumption around 9am among city users: {max_city_users:.2f} kWh', xy=(city_users_avg.idxmax(), max_city_users),
             xytext=(city_users_avg.idxmax(), max_city_users -18),
             fontsize=10)

plt.xticks(rotation=90)
plt.ylabel('Avg. Energy Consumption',  fontsize = 11)
plt.title('Avg Energy Consumption by User Type')
plt.legend()



#Number of Sessions by User Type
all_users= df[df['Station Name'].isin(['DCAS Storehouse', 'DPR Ocean Breeze', 'NYPD Cadet Academy'])]
city_users=df[df['Station Name'].isin(['DCAS Kent Ave', 'DSNY MN 2', 'DOT Harper St',
       'DSNY SI2', 'OCME 26th St.',
       'DEP Oakwood Beach WWTP', 'DSNY MN 05',
       'DEP Red Hook WWTP', 'DEP Newtown Creek WWTP',
       'DEP 26th Ward WWTP', "DEP Owl's Head WWTP"])]
all_users_avg= all_users.groupby('StartDate_Hour')['Transaction ID'].nunique()
city_users_avg= city_users.groupby('StartDate_Hour')['Transaction ID'].nunique()
hours=['12:00 AM','1:00 AM','2:00 AM','3:00 AM','4:00 AM','5:00 AM','6:00 AM','7:00 AM','8:00 AM','9:00 AM','10:00 AM'
       ,'11:00 AM','12:00 PM', '1:00 PM', '2:00 PM', '3:00 PM', '4:00 PM', '5:00 PM', '6:00 PM', '7:00 PM', '8:00 PM', '9:00 PM', '10:00 PM', '11:00 PM']
max_all_users=all_users_avg.max()  
max_city_users=city_users_avg.max()

all_users_avg= all_users_avg.reindex(hours)
city_users_avg= city_users_avg.reindex(hours)

plt.figure(figsize=(13,8))
plt.plot(all_users_avg.index, all_users_avg.values, label= 'All Users', color='darkcyan')
plt.plot(city_users_avg.index, city_users_avg.values, label= 'City Users', color='midnightblue')

plt.annotate(f'Max. Number of Sessions around 2pm: {max_all_users} (All Users)', xy=(all_users_avg.idxmax(), max_all_users),
             xytext=(all_users_avg.idxmax(), max_all_users +1 ),
             fontsize=10)
plt.annotate(f'Max. Number of Sessions around 12pm: {max_city_users:} (City Users)', xy=(city_users_avg.idxmax(), max_city_users),
             xytext=(city_users_avg.idxmax(), max_city_users -350),
             fontsize=10)

plt.xticks(rotation=90)
plt.ylabel('Avg. Energy Consumption',  fontsize = 11)
plt.title('Number of Sessions by User Type')
plt.legend(loc='upper left')



#Avg. Energy Consumption Monthly by User Type
all_users= df[df['Station Name'].isin(['DCAS Storehouse', 'DPR Ocean Breeze', 'NYPD Cadet Academy'])]
city_users=df[df['Station Name'].isin(['DCAS Kent Ave', 'DSNY MN 2', 'DOT Harper St',
       'DSNY SI2', 'OCME 26th St.',
       'DEP Oakwood Beach WWTP', 'DSNY MN 05',
       'DEP Red Hook WWTP', 'DEP Newtown Creek WWTP',
       'DEP 26th Ward WWTP', "DEP Owl's Head WWTP"])]
all_users_avg= all_users.groupby('StartDate_Month')['Energy (kWh)'].mean()
city_users_avg= city_users.groupby('StartDate_Month')['Energy (kWh)'].mean()

months= ['January', 'February', 'March', 'April', 'May', 'June',
        'July', 'August', 'September', 'October', 'November', 'December']

all_users_avg= all_users_avg.reindex(months)
city_users_avg= city_users_avg.reindex(months)

plt.figure(figsize=(10,6))
plt.plot(all_users_avg.index, all_users_avg.values, label= 'All Users', color='darkcyan')
plt.plot(city_users_avg.index, city_users_avg.values, label= 'City Users', color='midnightblue')

plt.xticks(rotation=90)
plt.ylabel('Avg. Energy Consumption per Month',  fontsize = 11)
plt.title('Avg. Energy Consumption Monthly by User Type')
plt.legend(loc='lower left')